In [0]:
from pyspark.sql import functions as F
import pandas as pd

print("Loading all tables (with FIXED crossborder - double underscore!)...")
print("="*60)

CATALOG = "curlybyte_solutions_rawdata_europe_grid_load"
GRID_SCHEMA = "european_grid_raw__v2"
WEATHER_SCHEMA = "european_weather_raw"

# Load crossborder_flows
crossborder_flows = spark.table(f"{CATALOG}.{GRID_SCHEMA}.crossborder_flows")
generation = spark.table(f"{CATALOG}.{GRID_SCHEMA}.generation")
installed_capacity = spark.table(f"{CATALOG}.{GRID_SCHEMA}.installed_capacity")
load_actual = spark.table(f"{CATALOG}.{GRID_SCHEMA}.load_actual")
load_forecast = spark.table(f"{CATALOG}.{GRID_SCHEMA}.load_forecast")
solar_forecast = spark.table(f"{CATALOG}.{GRID_SCHEMA}.solar_forecast")
wind_forecast = spark.table(f"{CATALOG}.{GRID_SCHEMA}.wind_forecast")
weather_hourly = spark.table(f"{CATALOG}.{WEATHER_SCHEMA}.weather_hourly")

print("✓ All tables loaded!")
print("\n" + "="*60)
print("Row counts:")
print(f"  crossborder_flows:   {crossborder_flows.count():>12,}")
print(f"  generation:          {generation.count():>12,}")
print(f"  installed_capacity:  {installed_capacity.count():>12,}")
print(f"  load_actual:         {load_actual.count():>12,}")
print(f"  load_forecast:       {load_forecast.count():>12,}")
print(f"  solar_forecast:      {solar_forecast.count():>12,}")
print(f"  wind_forecast:       {wind_forecast.count():>12,}")
print(f"  weather_hourly:      {weather_hourly.count():>12,}")

In [0]:
print("EXAMINING TABLE STRUCTURES")
print("="*60)

print("\n1. CROSSBORDER_FLOWS (FIXED):")
print(f"   Columns: {crossborder_flows.columns}")
display(crossborder_flows.limit(5))

print("\n2. LOAD_ACTUAL:")
print(f"   Columns: {load_actual.columns}")
display(load_actual.limit(3))

print("\n3. LOAD_FORECAST:")
print(f"   Columns: {load_forecast.columns}")
display(load_forecast.limit(3))

print("\n4. GENERATION:")
print(f"   Columns: {generation.columns}")
display(generation.limit(3))

print("\n5. SOLAR_FORECAST:")
print(f"   Columns: {solar_forecast.columns}")
display(solar_forecast.limit(3))

print("\n6. WIND_FORECAST:")
print(f"   Columns: {wind_forecast.columns}")
display(wind_forecast.limit(3))

print("\n7. WEATHER_HOURLY:")
print(f"   Columns: {weather_hourly.columns}")
display(weather_hourly.limit(3))

print("\n8. INSTALLED_CAPACITY:")
print(f"   Columns: {installed_capacity.columns}")
display(installed_capacity.limit(3))

In [0]:
print("COLUMN NAMES FOR EACH TABLE:")
print("="*60)
print(f"\n1. crossborder_flows: {crossborder_flows.columns}")
print(f"\n2. load_actual: {load_actual.columns}")
print(f"\n3. load_forecast: {load_forecast.columns}")
print(f"\n4. generation: {generation.columns}")
print(f"\n5. solar_forecast: {solar_forecast.columns}")
print(f"\n6. wind_forecast: {wind_forecast.columns}")
print(f"\n7. weather_hourly: {weather_hourly.columns}")
print(f"\n8. installed_capacity: {installed_capacity.columns}")

In [0]:
from pyspark.sql import functions as F

print("Creating master dataset by joining tables...")
print("="*60)

# Start with load_actual as base, rename 'index' to 'timestamp'
master = load_actual.withColumnRenamed('index', 'timestamp')
print(f"✓ Base: load_actual - {master.count():,} rows")

# Join load_forecast
master = master.join(
    load_forecast.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added load_forecast - {master.count():,} rows")

# Join generation
master = master.join(
    generation.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added generation - {master.count():,} rows")

# Join solar_forecast
master = master.join(
    solar_forecast.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added solar_forecast - {master.count():,} rows")

# Join wind_forecast
master = master.join(
    wind_forecast.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added wind_forecast - {master.count():,} rows")

print("\n" + "="*60)
print(f"MASTER DATASET: {master.count():,} rows, {len(master.columns)} columns")
print(f"Columns: {master.columns}")

# Show sample
print("\nSample data:")
display(master.limit(5))

In [0]:
from pyspark.sql import functions as F
import pandas as pd

print("DATA QUALITY ANALYSIS")
print("="*60)

# 1. Check rows per country
print("\n1. ROWS PER COUNTRY:")
country_counts = master.groupBy('country').count().orderBy(F.desc('count'))
display(country_counts)

# 2. Check which countries have generation data
print("\n2. COUNTRIES WITH vs WITHOUT GENERATION DATA:")
countries_with_gen = master.filter(
    F.col('Biomass__Actual_Aggregated').isNotNull()
).select('country').distinct().count()

total_countries = master.select('country').distinct().count()

print(f"   Total countries: {total_countries}")
print(f"   With generation data: {countries_with_gen}")
print(f"   Without generation data: {total_countries - countries_with_gen}")

# 3. Sample countries with generation data
print("\n3. SAMPLE COUNTRIES WITH GENERATION DATA:")
countries_gen_list = master.filter(
    F.col('Biomass__Actual_Aggregated').isNotNull()
).select('country').distinct().orderBy('country')
display(countries_gen_list)

# 4. Check missing percentages for key columns
print("\n4. MISSING VALUE PERCENTAGES:")
total_rows = master.count()

key_cols = ['Actual_Load', 'Forecasted_Load', 'Solar', 'Wind_Onshore', 
            'Nuclear__Actual_Aggregated', 'Solar__Actual_Aggregated']

missing_data = []
for col in key_cols:
    null_count = master.filter(F.col(col).isNull()).count()
    missing_pct = (null_count / total_rows) * 100
    missing_data.append({
        'column': col,
        'missing_pct': f"{missing_pct:.1f}%"
    })

missing_df = pd.DataFrame(missing_data)
print(missing_df)

In [0]:
print("CREATING TWO DATASET VERSIONS")
print("="*60)

# VERSION 1: ALL COUNTRIES - Basic features only (no generation)
print("\nVERSION 1: All Countries - Load + Forecasts Only")
print("-"*60)

master_v1 = master.select(
    'timestamp', 'country', 
    'Actual_Load', 'Forecasted_Load', 
    'Solar', 'Wind_Onshore'
).filter(
    F.col('Actual_Load').isNotNull()  # Remove rows with no load data
)

print(f"Rows: {master_v1.count():,}")
print(f"Columns: {len(master_v1.columns)}")
print(f"Countries: {master_v1.select('country').distinct().count()}")

# VERSION 2: 8 COUNTRIES - All features including generation
print("\nVERSION 2: 8 Countries - Complete Features")
print("-"*60)

countries_with_gen = ['AT', 'BE', 'CH', 'DE', 'ES', 'FR', 'IT', 'NL']
master_v2 = master.filter(
    (F.col('country').isin(countries_with_gen)) & 
    (F.col('Actual_Load').isNotNull())
)

print(f"Rows: {master_v2.count():,}")
print(f"Columns: {len(master_v2.columns)}")
print(f"Countries: {master_v2.select('country').distinct().count()}")

print("\n" + "="*60)
print("DECISION TIME:")
print("  Version 1: More data (all countries), fewer features")
print("  Version 2: Less data (8 countries), all features")
print("\nWhich version do you want to use for correlation analysis?")
print("(Recommendation: Version 2 for better insights)")

In [0]:
from pyspark.sql import functions as F
import pandas as pd

# Step 1: Load all tables
print("STEP 1: Loading all tables...")
print("="*60)

CATALOG = "curlybyte_solutions_rawdata_europe_grid_load"
GRID_SCHEMA = "european_grid_raw__v2"
WEATHER_SCHEMA = "european_weather_raw"

crossborder_flows = spark.table(f"{CATALOG}.{GRID_SCHEMA}.crossborder_flows")
generation = spark.table(f"{CATALOG}.{GRID_SCHEMA}.generation")
installed_capacity = spark.table(f"{CATALOG}.{GRID_SCHEMA}.installed_capacity")
load_actual = spark.table(f"{CATALOG}.{GRID_SCHEMA}.load_actual")
load_forecast = spark.table(f"{CATALOG}.{GRID_SCHEMA}.load_forecast")
solar_forecast = spark.table(f"{CATALOG}.{GRID_SCHEMA}.solar_forecast")
wind_forecast = spark.table(f"{CATALOG}.{GRID_SCHEMA}.wind_forecast")
weather_hourly = spark.table(f"{CATALOG}.{WEATHER_SCHEMA}.weather_hourly")

print("✓ All tables loaded!")

# Step 2: Join tables
print("\nSTEP 2: Joining tables with LEFT JOIN...")
print("="*60)

# Start with load_actual, rename 'index' to 'timestamp'
master = load_actual.withColumnRenamed('index', 'timestamp')
print(f"✓ Base: load_actual - {master.count():,} rows")

# Join load_forecast
master = master.join(
    load_forecast.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added load_forecast")

# Join generation
master = master.join(
    generation.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added generation")

# Join solar_forecast
master = master.join(
    solar_forecast.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added solar_forecast")

# Join wind_forecast
master = master.join(
    wind_forecast.withColumnRenamed('index', 'timestamp'),
    on=['timestamp', 'country'],
    how='left'
)
print(f"✓ Added wind_forecast")

print(f"\n✓ Master dataset created: {master.count():,} rows, {len(master.columns)} columns")

# Step 3: Clean the dataset
print("\nSTEP 3: Cleaning dataset...")
print("="*60)

# Remove rows with no actual load
master_clean = master.filter(F.col('Actual_Load').isNotNull())
print(f"✓ Filtered rows: {master_clean.count():,} rows remaining")

# Fill NULLs with 0 for generation columns (NULL = no capacity)
generation_cols = [col for col in master_clean.columns if 'Actual_Aggregated' in col or 'Actual_Consumption' in col]
print(f"✓ Filling {len(generation_cols)} generation columns with 0 (where NULL = no capacity)...")

for col in generation_cols:
    master_clean = master_clean.fillna(0, subset=[col])

print("\n" + "="*60)
print("FINAL MASTER DATASET:")
print(f"  Rows: {master_clean.count():,}")
print(f"  Columns: {len(master_clean.columns)}")
print(f"  Countries: {master_clean.select('country').distinct().count()}")

# Show sample
print("\nSample data:")
display(master_clean.limit(5))

In [0]:
print("Converting ALL data to Pandas (no sampling)...")
print("="*60)

# Convert entire dataset
print(f"Converting {master_clean.count():,} rows to Pandas...")
print("This may take 1-3 minutes...")

import time
start_time = time.time()

pandas_df = master_clean.toPandas()

elapsed_time = time.time() - start_time

print(f"\n✓ SUCCESS! Pandas DataFrame created in {elapsed_time:.1f} seconds")
print(f"  Shape: {pandas_df.shape[0]:,} rows × {pandas_df.shape[1]} columns")
print(f"  Memory usage: {pandas_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\nFirst 5 rows:")
print(pandas_df.head())

print("\nData types:")
print(pandas_df.dtypes)

print("\nSummary Statistics:")
print(pandas_df.describe())

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

print("STEP 7: CORRELATION ANALYSIS")
print("="*60)

# Select only numeric columns for correlation
numeric_cols = pandas_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(f"\nNumeric columns: {len(numeric_cols)}")
print(numeric_cols)

# Calculate correlation matrix
print("\nCalculating correlation matrix...")
correlation_matrix = pandas_df[numeric_cols].corr()

print(f"✓ Correlation matrix calculated: {correlation_matrix.shape}")

# Show correlations with Actual_Load (our target)
print("\n" + "="*60)
print("CORRELATIONS WITH ACTUAL_LOAD (sorted by strength):")
print("="*60)
load_correlations = correlation_matrix['Actual_Load'].sort_values(ascending=False)
print(load_correlations)

# Visualize correlation heatmap
print("\nCreating correlation heatmap...")
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - European Power Grid Features', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [0]:
# Create a cleaner visualization of top correlations
print("\nTOP 10 CORRELATIONS WITH ACTUAL_LOAD:")
print("="*60)

top_correlations = load_correlations.drop('Actual_Load').head(10)

plt.figure(figsize=(10, 6))
top_correlations.plot(kind='barh', color='steelblue')
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.title('Top 10 Features Correlated with Actual Load', fontsize=14)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
print("STEP 8: SAVING DATASET")
print("="*60)

# Option 1: Save as Delta table (BEST for Databricks)
print("\n1. Saving as Delta table...")
spark_df = spark.createDataFrame(pandas_df)
spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.default.power_grid_master")
print("   ✓ Saved: workspace.default.power_grid_master")

# Option 2: Save correlation matrix as Delta table
print("\n2. Saving correlation matrix as Delta table...")
corr_df = correlation_matrix.reset_index()
corr_df.columns = ['feature'] + list(corr_df.columns[1:])
corr_spark = spark.createDataFrame(corr_df)
corr_spark.write.format("delta").mode("overwrite").saveAsTable("workspace.default.correlation_matrix")
print("   ✓ Saved: workspace.default.correlation_matrix")

# Verify the saves
print("\n3. Verifying saved tables...")
master_count = spark.table("workspace.default.power_grid_master").count()
corr_count = spark.table("workspace.default.correlation_matrix").count()
print(f"   ✓ power_grid_master: {master_count:,} rows")
print(f"   ✓ correlation_matrix: {corr_count} rows")

print("\n" + "="*60)
print("SUMMARY - Delta Tables Saved:")
print("="*60)
print(f"  Dataset: {pandas_df.shape[0]:,} rows × {pandas_df.shape[1]} cols")
print(f"  Memory: {pandas_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"  Date range: {pandas_df['timestamp'].min()} to {pandas_df['timestamp'].max()}")
print(f"  Countries: {pandas_df['country'].nunique()}")
print("\n✓ All tables saved successfully!")

print("\n" + "="*60)
print("HOW TO LOAD LATER:")
print("="*60)
print("""
# Load main dataset:
df = spark.table('workspace.default.power_grid_master').toPandas()

# Load correlation matrix:
corr = spark.table('workspace.default.correlation_matrix').toPandas()
corr = corr.set_index('feature')

# Or use in Spark directly:
spark_df = spark.table('workspace.default.power_grid_master')
""")

In [0]:
# Show the exact location of your tables
print("LOCATING YOUR SAVED TABLES")
print("="*60)

# Method 1: In the Databricks UI
print("\n📁 METHOD 1: Databricks UI (Left Sidebar)")
print("-"*60)
print("1. Click 'Catalog' icon in the left sidebar")
print("2. Navigate to: workspace → default")
print("3. You'll see:")
print("   ✓ power_grid_master")
print("   ✓ correlation_matrix")

# Method 2: Show tables via SQL
print("\n📊 METHOD 2: List tables programmatically")
print("-"*60)
display(spark.sql("SHOW TABLES IN workspace.default"))

# Method 3: Get detailed table info
print("\n📋 METHOD 3: Detailed table information")
print("-"*60)

print("\n1. POWER_GRID_MASTER:")
spark.sql("DESCRIBE EXTENDED workspace.default.power_grid_master").show(30, truncate=False)

print("\n2. CORRELATION_MATRIX:")
spark.sql("DESCRIBE EXTENDED workspace.default.correlation_matrix").show(30, truncate=False)

# Show physical location
print("\n💾 PHYSICAL STORAGE LOCATION:")
print("-"*60)
location_df = spark.sql("""
    DESCRIBE EXTENDED workspace.default.power_grid_master
""").filter("col_name = 'Location'")
display(location_df)

print("\n✓ Your tables are stored in the Databricks workspace!")
print("✓ They persist across sessions and won't be deleted unless you explicitly drop them")

In [0]:
print("ANALYZING DATA TO DEFINE GRID FAILURES")
print("="*60)

# Load your saved dataset
df = spark.table('workspace.default.power_grid_master').toPandas()

# Option 1: Check if we can calculate total generation
print("\n1. CHECKING TOTAL GENERATION vs LOAD:")
generation_cols = [col for col in df.columns if 'Actual_Aggregated' in col]
print(f"   Generation columns available: {len(generation_cols)}")

# Calculate total generation
df['Total_Generation'] = df[generation_cols].sum(axis=1)
df['Generation_Deficit'] = df['Actual_Load'] - df['Total_Generation']

print(f"   Mean generation deficit: {df['Generation_Deficit'].mean():.2f}")
print(f"   % of time with deficit: {(df['Generation_Deficit'] > 0).sum() / len(df) * 100:.2f}%")

# Option 2: Forecast deviation
print("\n2. CHECKING FORECAST DEVIATIONS:")
df['Forecast_Error'] = abs(df['Actual_Load'] - df['Forecasted_Load'])
df['Forecast_Error_Pct'] = (df['Forecast_Error'] / df['Forecasted_Load']) * 100

print(f"   Mean forecast error: {df['Forecast_Error'].mean():.2f} MW")
print(f"   Mean forecast error %: {df['Forecast_Error_Pct'].mean():.2f}%")
print(f"   95th percentile error: {df['Forecast_Error_Pct'].quantile(0.95):.2f}%")

# Large errors (potential failures)
large_errors = df[df['Forecast_Error_Pct'] > 10]
print(f"   Rows with >10% error: {len(large_errors)} ({len(large_errors)/len(df)*100:.2f}%)")

# Option 3: Rapid load changes
print("\n3. CHECKING RAPID LOAD CHANGES:")
df_sorted = df.sort_values(['country', 'timestamp'])
df_sorted['Load_Change'] = df_sorted.groupby('country')['Actual_Load'].diff()
df_sorted['Load_Change_Pct'] = (df_sorted['Load_Change'] / df_sorted['Actual_Load']) * 100

print(f"   Mean hourly load change: {df_sorted['Load_Change_Pct'].abs().mean():.2f}%")
print(f"   95th percentile change: {df_sorted['Load_Change_Pct'].abs().quantile(0.95):.2f}%")

rapid_changes = df_sorted[df_sorted['Load_Change_Pct'].abs() > 20]
print(f"   Rows with >20% change: {len(rapid_changes)} ({len(rapid_changes)/len(df_sorted)*100:.2f}%)")

# Summary
print("\n" + "="*60)
print("RECOMMENDATION FOR FAILURE DEFINITION:")
print("="*60)
display(df[['timestamp', 'country', 'Actual_Load', 'Forecasted_Load', 'Total_Generation', 
            'Forecast_Error_Pct', 'Generation_Deficit']].head(10))

In [0]:
print("CHECKING RENEWABLE DATA AVAILABILITY")
print("="*60)

# Load your dataset
df = spark.table('workspace.default.power_grid_master').toPandas()

# Check renewable features
renewable_features = [
    'Solar_Forecast',
    'Wind_Onshore_Forecast', 
    'Solar__Actual_Aggregated',
    'Wind_Onshore__Actual_Aggregated',
    'Wind_Offshore__Actual_Aggregated',
    'Hydro_Water_Reservoir__Actual_Aggregated',
    'Hydro_Run_of_river_and_poundage__Actual_Aggregated'
]

print("\n1. RENEWABLE DATA AVAILABILITY:")
print("-"*60)
for col in renewable_features:
    if col in df.columns:
        non_null = df[col].notna().sum()
        non_zero = (df[col] > 0).sum()
        pct_available = (non_null / len(df)) * 100
        pct_active = (non_zero / len(df)) * 100
        print(f"{col}:")
        print(f"  Non-null: {non_null:,} ({pct_available:.1f}%)")
        print(f"  Non-zero: {non_zero:,} ({pct_active:.1f}%)")
        print()

# Check forecast vs actual correlation
print("\n2. FORECAST VS ACTUAL CORRELATIONS:")
print("-"*60)
if 'Solar_Forecast' in df.columns and 'Solar__Actual_Aggregated' in df.columns:
    corr_solar = df[['Solar_Forecast', 'Solar__Actual_Aggregated']].corr().iloc[0, 1]
    print(f"Solar Forecast ↔ Solar Actual: {corr_solar:.3f}")

if 'Wind_Onshore_Forecast' in df.columns and 'Wind_Onshore__Actual_Aggregated' in df.columns:
    corr_wind = df[['Wind_Onshore_Forecast', 'Wind_Onshore__Actual_Aggregated']].corr().iloc[0, 1]
    print(f"Wind Forecast ↔ Wind Actual: {corr_wind:.3f}")

# Countries with renewable data
print("\n3. COUNTRIES WITH RENEWABLE DATA:")
print("-"*60)
countries_with_solar = df[df['Solar__Actual_Aggregated'] > 0]['country'].nunique()
countries_with_wind = df[df['Wind_Onshore__Actual_Aggregated'] > 0]['country'].nunique()
total_countries = df['country'].nunique()

print(f"Countries with Solar data: {countries_with_solar}/{total_countries}")
print(f"Countries with Wind data: {countries_with_wind}/{total_countries}")

# Alternative features we DO have
print("\n4. FEATURES WE DEFINITELY HAVE:")
print("-"*60)
guaranteed_features = [
    'Forecasted_Load',
    'Actual_Load',
    'timestamp',
    'country'
]

for col in guaranteed_features:
    if col in df.columns:
        print(f"✅ {col} - 100% available")

print("\n" + "="*60)
print("CONCLUSION:")
print("="*60)